#Optimizer Methods Testing

This is notebook for testing the workflow of the optimizer object in MDSAPT. Optimizer was developed to solve the issue of residues having an unbalanced spin when drawn directly from protein backbone. It does this over a few steps addressing the different issues. 

In [1]:
import os

import numpy as np

import MDAnalysis as mda

from pdbfixer import PDBFixer

from simtk.openmm.app import PDBFile

import nglview as nv

from mdsapt.reader import InputReader

2021-12-13 11:10:09,110 MDSAPT       INFO     MDSAPT 0+untagged.30.g1cdfbae.dirty starting
2021-12-13 11:10:09,112 MDSAPT       INFO     Copyright (c) 2021 Alia Lescoulie
2021-12-13 11:10:09,112 MDSAPT       INFO     Released under MIT Licence


In [2]:
In = InputReader(os.path.join(os.getcwd(), 'mdsapt', 'tests', 'testing_resources', 'test_input.yaml'))

unv = mda.Universe(In.top_path, In.trj_path)

'pH' Error in trajectory settings


##1. Loading the Residue

In this case ARG is the residue. Both the amino and carboxyl side of the residue are missing protons.

In [3]:
# Selection prior to fixing

res = unv.select_atoms('resid 2')

nv.show_mdanalysis(res)

NGLWidget(max_frame=97)

##2. Replacing missing Amino Protons
The file is loaded into PDBFixer from opennm and amino protons are reattached. 

In [5]:
res.write('resid.pdb', file_format='PDB') # Saving residue
fixer = PDBFixer(filename='resid.pdb')
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingHydrogens(7) # Adding protons at pH value
PDBFile.writeFile(fixer.topology, fixer.positions, open('resid_fixed.pdb', 'w'))


res_fixed = mda.Universe('resid_fixed.pdb')
resid: mda.AtomGroup = res_fixed.select_atoms("resname *")
resid.guess_bonds()
nv.show_mdanalysis(resid)

/Users/alia/opt/anaconda3/envs/MDSAPT/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/Users/alia/opt/anaconda3/envs/MDSAPT/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/MDSAPT/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/MDSAPT/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/MDSAPT/lib/python3.8/site

NGLWidget()

##3. Adding New Proton

The carbon on the carboxyl end of the residue only has three bonds when removed from the backbone. Rather than adding a hydroxyl group, a proton is added to the end as a 4th bond.

In [11]:
bkbone_fixed = mda.Universe('resid_fixed.pdb') # Load PDBRepair segid
bkbone_resid = bkbone_fixed.select_atoms("backbone") # Load backbone
ha = bkbone_fixed.select_atoms('name HA') # Remove proton on Alpha carbon
bkbone_resid = bkbone_resid - ha # Remove proton on Alpha carbon
carbon = bkbone_resid.select_atoms('name C') 
pos = carbon.atoms[0].position # Get postion of end carbon
h = mda.Universe.empty(n_atoms=bkbone_resid.n_atoms + 1, trajectory=True)
h.add_TopologyAttr("masses", [x for x in bkbone_resid.masses] + [1])
h.add_TopologyAttr("name", [x for x in bkbone_resid.names] + ['H'])
bkbone_pos = bkbone_resid.positions
h.atoms.positions = np.row_stack((bkbone_pos, np.array([pos[0] - np.cos(np.pi/6), pos[1] - np.sin(np.pi/6), pos[2]])))
nv.show_mdanalysis(h)


/Users/alia/opt/anaconda3/envs/MDSAPT/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:429: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"
/Users/alia/opt/anaconda3/envs/MDSAPT/lib/python3.8/site-packages/MDAnalysis/core/universe.py:433: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/Users/alia/opt/anaconda3/envs/MDSAPT/lib/python3.8/site-packages/MDAnalysis/core/universe.py:439: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


NGLWidget()

##4. Optimizing New Bond

Psi4 is employed to optimize the length of the newly added C-H bond.

In [19]:
import psi4
coords: str = ''
freeze_list = ''
opt_settings: dict = {'reference': 'rhf'} # Psi4 settings

# Getting coords and settings
for n in range(len(h.atoms)):
    atom = h.atoms[n]
    coords += f"\n{atom.name[0]} {atom.position[0]} {atom.position[1]} {atom.position[1]}"
    if not atom.name == 'H':
        freeze_list += f"\n{n + 1} xyz" # Freezing bonds besides H
print(freeze_list)
print(coords)

mol = psi4.geometry(coords)



1 xyz
2 xyz
3 xyz
4 xyz

N 11.840999603271484 6.642000198364258 6.642000198364258
C 11.414999961853027 5.434000015258789 5.434000015258789
C 10.770999908447266 5.855000019073486 5.855000019073486
O 11.35200023651123 6.664000034332275 6.664000034332275
H 9.904974937438965 5.355000019073486 5.355000019073486


In [25]:
psi4.set_memory('8GB')
psi4.set_options(opt_settings)
psi4.optimize('scf/dz', freeze_list=freeze_list, opt_coordinates='cartesian', molecule=mol)


  Memory set to   7.451 GiB by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on Alias-MBP
*** at Mon Dec 13 14:27:17 2021

   => Loading Basis Set <=

    Name: DZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry N          line    85 file /Users/alia/opt/anaconda3/envs/MDSAPT/share/psi4/basis/dz.gbs 
    atoms 2-3 entry C          line    63 file /Users/alia/opt/anaconda3/envs/MDSAPT/share/psi4/basis/dz.gbs 
    atoms 4   entry O          line   107 file /Users/alia/opt/anaconda3/envs/MDSAPT/share/psi4/basis/dz.gbs 
    atoms 5   entry H          line    12 file /Users/alia/opt/anaconda3/envs/MDSAPT/share/psi4/basis/dz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference

-205.25107679840085

			--------------------------
			 OPTKING Finished Execution 
			--------------------------

    Final optimized geometry and variables:
    Molecular point group: cs
    Full point group: Cs

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

    N            1.202666751483    -0.834343464544     0.000000000000
    C           -1.656463857768    -0.271350061996     0.000000000000
    C           -0.650312123551     0.597844377565     0.000000000000
    O            0.735414765323     0.382239515889     0.000000000000
    H           -0.915452682415     1.638729956691     0.000000000000

	Removing binary optimization data file.
	Cleaning optimization helper files.
